In [1]:
import folium
import jinja2
import random
import pandas as pd
import requests
import folium
import polyline
from tqdm import tqdm

In [2]:
location_center=[39.64688,27.88862]

m = folium.Map(location_center, zoom_start=8.5)
display(m)

In [3]:
numberofLoc = 20
coordinates=[None] * numberofLoc
for i in range(0, numberofLoc):
    coordinates[i]=[random.uniform(39.347,39.743),random.uniform(27.2236, 28.3194)]


In [16]:
from numpy import asarray
from numpy import savetxt
data = asarray(coordinates)
savetxt('CoordinatesforGeneticAlgorithm.csv', data, delimiter=',')

df=pd.DataFrame(coordinates)
df.to_csv('Kordinatlar.csv',index=False) 

In [5]:
print(data)

[[39.62094497 28.31307225]
 [39.70846806 27.90552465]
 [39.72126291 27.36809935]
 [39.72796325 27.59002781]
 [39.71119864 27.81819135]
 [39.65795657 27.78339323]
 [39.66749185 27.89962301]
 [39.60783323 27.67774131]
 [39.67257088 27.76411155]
 [39.47055783 27.37847627]
 [39.57897467 27.99207129]
 [39.39549766 28.24092665]
 [39.37840075 27.25944496]
 [39.56323683 27.31888073]
 [39.74198422 27.979703  ]
 [39.50829483 28.00298492]
 [39.72505889 28.06697364]
 [39.42690718 27.27340335]
 [39.68907785 28.07618403]
 [39.56324697 28.16493981]]


In [6]:
location_center=[39.64688,27.88862]
locations = coordinates

m = folium.Map(location_center, zoom_start=8.5)
for location in locations:
    folium.Marker(
        location=location,
        popup = f'<input type="text" value="{location[0]}, {location[1]}" id="myInput"><button onclick="myFunction()">Copy location</button>'
    ).add_to(m)
  
display(m)



In [7]:
#İki noktanın kordinatlarını alarak geriye aralarındaki mesafeyi döndüren metot
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    #OSRM Api ile bağlantı kuruluyor.
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    #api'ın bizden istediği parametler doğrultusunda bir url oluşturuyoruz. Driving yazmamızın sebebi Sürüş rotalarını hesaplamak, 
    #bisiklet için hesaplamak istersek bike yazabiliriz.
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc) 
    if r.status_code!= 200:
        return {}
  
    #Başlangıç ve bitiş noktası olarak aldığı parametreleri kullanarak iki nokta arasındaki mesafeyi döndürüyor 
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = distance

    return out


In [8]:
distMatrix = [None]*numberofLoc

print(distMatrix)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [9]:
liste=[]
#İç içe 2 döngü oluşturularak 20x20 büyüklüğünde uzaklık matrisi elde ediliyor. Döngü teker teker bütün lokasyonlar arasındaki sürüş mesafelerini hesaplayacak
#olan get_route metotuna lokasyonları gönderiyor ve karşılığında sürüş mesafelerini elde ederek bu verileri matrise kaydediyor.
for i in tqdm(range(0, numberofLoc)):
    for j in range(0, numberofLoc):
        pickup_lat, pickup_lon = coordinates[i]
        dropoff_lat, dropoff_lon = coordinates[j]
        test_route = get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
        distMatrix[j]=test_route
    liste.append(distMatrix)
    distMatrix = [None]*numberofLoc
        



100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:21<00:00, 10.09s/it]


In [10]:
df=pd.DataFrame(liste)
df.to_csv('uzaklık matrisi.csv',index=False,header=False)

In [11]:
import numpy as np

array_mat = np.loadtxt('uzaklık matrisi.csv', delimiter=',')
print(array_mat)

[[     0.   51704.6 111068.5  95356.   61506.1  61400.3  47459.7  71910.1
   64599.  108421.8  44954.1  72476.3 128862.  117971.2  56696.4  69446.
   32533.6 121338.8  31318.5  34229.9]
 [ 52344.7      0.   71825.7  56113.2  22263.3  22157.6   6232.4  32667.4
   25356.3  69179.   24540.8  62704.4  89619.2  78728.4  26594.4  30203.3
   32362.9  82096.1  25244.1  43204. ]
 [111159.1  70564.7      0.   41334.1  64047.3  53934.4  64541.3  54254.4
   56245.2  49501.6  77504.3 115667.9  69941.8  42155.9  82004.9  83166.8
   89630.4  62418.6  84058.5 102018.4]
 [ 95437.   54842.6  41334.1      0.   48325.2  38212.3  48819.2  38532.3
   40523.1  44092.4  61782.2  99945.8  64532.6  53641.7  66282.8  67444.7
   73908.3  57009.4  68336.4  86296.3]
 [ 61439.9  20845.5  63913.6  48201.1      0.   14245.5  14822.1  24755.2
   13158.9  61266.9  27785.1  65948.7  81707.1  70816.3  35689.7  33447.6
   41458.1  74183.9  34339.3  52299.2]
 [ 62260.6  21666.2  53790.   38077.5  15148.8      0.   15642.8  